In [3]:
#from datetime import datetime
#import pytz
#import json
#import urllib
import numpy as np
import pandas as pd
from datetime import datetime, timedelta

from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource
from bokeh.models import HoverTool, TapTool, DatetimeTickFormatter, DateFormatter
from bokeh.tile_providers import get_provider, Vendors
from scipy.spatial.distance import cdist
#from bokeh.models.renderers import GlyphRenderer
#from bokeh.models.widgets import DataTable, TableColumn, Slider, Dropdown
from bokeh.layouts import layout, column, row, widgetbox
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

In [4]:
base_url = 'http://meteo145.uibk.ac.at/'
station = 'innsbruck'
time = str(3)

url = base_url + station + '/' + time

In [10]:
df = pd.read_json(url)
df['time'] = [datetime(1970, 1, 1) + timedelta(milliseconds=ds) for ds in df['datumsec']]
df = df.set_index('time')
df = df.drop(columns='datumsec')
df.head()

,rr,dd,tp,p,tl,so,ff
time,,,,,,,
2019-10-21 08:00:00,0.0,233,9.3,948.2,13.8,0.000000,2.7
2019-10-21 08:10:00,0.0,242,9.2,948.4,13.9,0.000000,2.3
2019-10-21 08:20:00,0.0,260,9.6,948.6,14.5,0.000000,2.6
2019-10-21 08:30:00,0.0,244,10.7,948.6,16.8,0.000000,0.8
2019-10-21 08:40:00,0.0,196,10.3,948.7,18.0,7.483333,3.6


In [13]:
for column in df.columns:
    df[df[column] == -99.9] = np.nan


In [55]:
def merc(lat, lon):
    """Convert latitude and longitude into mercator's x and y position
    Parameters
    ----------
    lon : longitude
    lat : latitude
    """
    r_major = 6378137.000
    x = r_major * np.radians(lon)
    scale = x/lon
    y = 180.0/np.pi * np.log(np.tan(np.pi/4.0 + 
        lat * (np.pi/180.0)/2.0)) * scale
    return (x, y)

In [59]:
stations = pd.DataFrame({'lat':[47.26], 'lon':[11.45]}, index=['innsbruck'])
[stations['x'], stations['y']]= merc(stations['lat'],stations['lon'])
stations

,lat,lon,x,y
innsbruck,47.26,11.45,1.274608e+06,5.984616e+06


In [64]:
tile_provider = get_provider(Vendors.CARTODBPOSITRON)
map_tools = 'box_zoom,pan,save,hover,reset,tap,wheel_zoom'
map_plot = figure(x_range=(1108137, 1417582), y_range=(5895123 , 6088551), plot_width=600, plot_height=350,
           x_axis_type="mercator", y_axis_type="mercator", tools=map_tools)#, sizing_mode="scale_both")
map_plot.add_tile(tile_provider)
map_plot.circle(x="x", y="y", size=15, fill_color="blue", fill_alpha=0.4, source=stations);
hover_map = map_plot.select(dict(type=HoverTool))
hover_map.tooltips = [("Stationsname", "@index")]
hover_map.mode = 'mouse'
show(map_plot)

In [ ]:
p1.figure()
